In [1]:
import pandas as pd
import rdflib as rdf
from SPARQLWrapper import SPARQLWrapper, JSON
import math
import numpy as np

In [ ]:
df_ml = pd.read_csv('/home/ignacio/Datasets/ml-latest-small/ratings.csv')
df_ml.head()

In [ ]:
df_ml_links = pd.read_csv('/home/ignacio/Datasets/ml-latest-small/links.csv')
df_ml_links.head()


## Add director link for dbpedia

### Clean title for dbpedia

In [ ]:
df_ml_movies = pd.read_csv('/home/ignacio/Datasets/ml-latest-small/movies.csv')
df_ml_movies.head()

In [ ]:
s_titles = df_ml_movies['title']
s_titles.head()

In [ ]:
s_titles = s_titles.str.split(r'(').str[0]

In [ ]:
s_titles = s_titles.str[:-1]

In [ ]:
s_titles.head()

In [ ]:
len(s_titles)

In [ ]:
s_year = df_ml_movies['title'].str.extract(r'(\d\d\d\d)')
s_year.head()

In [ ]:
s_year = s_year.str[:-1]
s_year.head()

In [ ]:
s_titles_The = s_titles.str.findall(r'.+, The')
s_titles_The = s_titles[s_titles_The.str.len() > 0]
s_titles_The.head()

In [ ]:
s_titles_The = s_titles.str.findall(r'.+, An$')
s_titles_The = s_titles[s_titles_The.str.len() > 0]
s_titles_The.head()

In [ ]:
len(s_titles_The)

In [ ]:
s_titles_The = s_titles_The.str.replace(r'.+, The',lambda x: x.group(0)[-3:]+ ' ' +x.group()[:-5])
s_titles_The.head()

In [ ]:
s_titles_The = s_titles_The.str.replace(r'.+, An$',lambda x: x.group(0)[-2:]+ ' ' +x.group()[:-4])
s_titles_The = s_titles_The.str.replace(r'(\s|,\s)', '_')
s_titles_The.head()

In [ ]:
s_titles_The = s_titles_The.str.replace(r'(\s|,\s)', '_')

In [ ]:
s_titles_The.head()

In [ ]:
s_titles = s_titles.str.replace(r'.+, The$',lambda x: x.group(0)[-3:]+ ' ' +x.group()[:-5])
s_titles = s_titles.str.replace(r'.+, Les$',lambda x: x.group(0)[-3:]+ ' ' +x.group()[:-5])
s_titles = s_titles.str.replace(r'.+, A$',lambda x: x.group(0)[-1:]+ ' ' +x.group()[:-3])
s_titles = s_titles.str.replace(r'.+, An$',lambda x: x.group(0)[-2:]+ ' ' +x.group()[:-4])
s_titles = s_titles.str.replace(r'.+, Le$',lambda x: x.group(0)[-2:]+ ' ' +x.group()[:-4])
s_titles = s_titles.str.replace(r'.+, La$',lambda x: x.group(0)[-2:]+ ' ' +x.group()[:-4])
#s_titles = s_titles.str.replace(r'(\s|,\s)', '_')
s_titles.head()

In [ ]:
s_titles[0]

In [ ]:
len(s_titles)

In [ ]:
df_ml_movies['title_mod'] = s_titles
df_ml_movies['year'] = s_year
df_ml_movies.head()

In [ ]:
df_ml_movies.to_csv('/home/ignacio/Datasets/ml-latest-small/movies_mod.csv', index=False)

### Find director on dbpedia

In [ ]:
df_ml_movies = pd.read_csv('/home/ignacio/Datasets/ml-latest-small/movies_mod.csv')
df_ml_movies.head()

In [ ]:
df_ml_movies['year'].dtype

In [ ]:
df_ml_movies['year'] = df_ml_movies['year'].fillna(method='ffill')

In [ ]:
df_ml_movies['year'] = df_ml_movies['year'].astype('int64')
df_ml_movies['year'] = df_ml_movies['year'].astype('str')
df_ml_movies.head()

#### Change outliers

In [ ]:
df_ml_movies['title_mod'] = df_ml_movies['title_mod'].str.replace('La Femme Nikita', 'Nikita')

In [ ]:
df_ml_movies = df_ml_movies[df_ml_movies['title_mod'] != '11\'09"01 - September 11']

In [ ]:
df_ml_movies = df_ml_movies[df_ml_movies['title_mod'] != '"Great Performances" Cats']

In [ ]:
df_ml_movies = df_ml_movies.drop([7269])

In [ ]:
df_ml_movies.head()

In [ ]:
df_ml_movies.to_csv('/home/ignacio/Datasets/ml-latest-small/movies_mod.csv', index=False)

#### DBpedia query

In [ ]:
sparql = SPARQLWrapper("http://dbpedia.org/sparql")

In [ ]:
title = df_ml_movies['title_mod'][0]

In [ ]:
title

In [ ]:
def find_director( row ):
    
    sparql = SPARQLWrapper("http://dbpedia.org/sparql")
    
    stm = """
        PREFIX rdfs: <http://www.w3.org/2000/01/rdf-schema#>
        PREFIX dbr: <http://dbpedia.org/resource/>
        PREFIX dbo: <http://dbpedia.org/ontology/>
        SELECT DISTINCT ?directorName
        WHERE {
           ?f a dbo:Film ;
           rdfs:label ?label .
           ?f dbo:abstract ?abs .
           ?f dbo:director ?director .
           ?director foaf:name ?directorName.
         FILTER (contains(str(?label),\"""" + row['title_mod'] + """\"))
         FILTER (lang(?label) = 'en')
         FILTER (contains(str(?abs),\"""" + row['year'] + "\")) }" 
         
    sparql.setQuery(stm)
    sparql.setReturnFormat(JSON)
    results = sparql.query().convert()
    director = []
    for result in results["results"]["bindings"]:
        director.append(result["directorName"]["value"])
    return director

In [ ]:
df_test = df_ml_movies.iloc[7266:7272]
df_test = df_test.drop([7269])
df_test

In [ ]:
df_test['director'] = df_test.apply(find_director, axis=1)
df_test.head()

In [ ]:
directors = df_test.iloc[0]['director']

In [ ]:
len(directors)

In [ ]:
frame = 1000
max_complete_frames = math.floor(len(df_ml_movies)/frame)
max_complete_frames

In [ ]:
for i in np.arange(7,max_complete_frames):
    df_test = df_ml_movies.loc[i*frame : (i+1)*frame]
    df_test['director'] = df_test.apply(find_director, axis=1)
    df_test.to_csv('/home/ignacio/Datasets/ml-latest-small/movies_mod_'+ str(i) +'.csv', index=False)
    print('Iteration ' + str(i) + ' finished')

if len(df_ml_movies) % frame != 0:
    df_test = df_ml_movies.loc[max_complete_frames*frame : len(df_ml_movies) ]
    df_test['director'] = df_test.apply(find_director, axis=1)
    df_test.to_csv('/home/ignacio/Datasets/ml-latest-small/movies_mod_'+ str(max_complete_frames + 1) +'.csv', index=False)

#### Join Results

In [ ]:
df_ml_movies_directors = pd.read_csv('/home/ignacio/Datasets/ml-latest-small/movies_mod_0.csv')

for i in np.arange(1, max_complete_frames+1):
    df = pd.read_csv('/home/ignacio/Datasets/ml-latest-small/movies_mod_'+ str(i) +'.csv')
    df_ml_movies_directors = pd.concat([df_ml_movies_directors, df], ignore_index=True)

print(len(df_ml_movies_directors))

In [ ]:
df_ml_movies_directors.to_csv('/home/ignacio/Datasets/ml-latest-small/movies_directors.csv', index=False)

## Find directors influences

#### Clean director format

In [ ]:
df_ml_movies_directors = pd.read_csv('/home/ignacio/Datasets/ml-latest-small/movies_directors.csv')

In [ ]:
df_ml_movies_directors.head()

In [ ]:
df_directors = df_ml_movies_directors['director']
df_directors.head()

In [ ]:
len(df_directors)

In [ ]:
df_directors = df_directors.str.replace(r'[\[\]\'\']', '')
df_directors = df_directors.str.split(r',\s')
df_directors.head()

In [ ]:
l_directors = list(df_directors)
l_directors_unified = [x for d in l_directors for x in d]
l_directors_unified[0:10]

In [ ]:
df_directors = pd.Series(data=l_directors_unified, name='directors')
df_directors.head()

In [ ]:
df_directors = df_directors.drop_duplicates()
len(df_directors)

In [ ]:
df_directors.to_csv('/home/ignacio/Datasets/ml-latest-small/directors.csv', index=False)

#### DBpedia query

In [ ]:
df_directors = pd.read_csv('/home/ignacio/Datasets/ml-latest-small/directors.csv')
df_directors = df_directors['Director']
df_directors.head()

In [ ]:
df_directors['Director'] = df_directors['Director'].str.replace('\"', '')
df_directors.head()

In [ ]:
df_directors = df_directors.drop([499,500])
df_directors.head()

In [ ]:
len(df_directors)

In [ ]:
sparql = SPARQLWrapper("http://dbpedia.org/sparql")

In [ ]:
def find_influence( row ):
    
    sparql = SPARQLWrapper("http://dbpedia.org/sparql")
    
    stm = """
        SELECT DISTINCT ?Aname 
        where {
          ?A dbo:influenced ?B.
          ?A a dbo:Person.
          ?B a dbo:Person.
          ?B foaf:name ?Bname .
          ?A foaf:name ?Aname 
          FILTER(regex(?Bname, \""""+ row +"""\"))
         }"""
         
    sparql.setQuery(stm)
    sparql.setReturnFormat(JSON)
    results = sparql.query().convert()
    director = []
    for result in results["results"]["bindings"]:
        director.append(result["Aname"]["value"])
    return director

In [ ]:
frame = 1000
max_complete_frames = math.floor(len(df_directors)/frame)
max_complete_frames

In [ ]:
for i in np.arange(0, max_complete_frames):
    df_test = df_directors.loc[i*frame : (i+1)*frame]
    df_test_influenced = df_test.apply(find_influence)
    df_to_save = pd.DataFrame({'Director':df_test, 'Influence':df_test_influenced})
    df_to_save.to_csv('/home/ignacio/Datasets/ml-latest-small/ml-directors-influence_'+ str(i) +'.csv', index=False)
    print("Iteration " + str(i) )
    
if len(df_directors) % frame != 0:
    df_test = df_directors.loc[max_complete_frames*frame : len(df_directors) ]
    df_test_influenced = df_test.apply(find_influence)
    df_to_save = pd.DataFrame({'Director':df_test, 'Influence':df_test_influenced})
    df_to_save.to_csv('/home/ignacio/Datasets/ml-latest-small/ml-directors-influence_'+ str(max_complete_frames ) +'.csv', index=False)

#### Join Results

In [ ]:
df_ml_influences = pd.read_csv('/home/ignacio/Datasets/ml-latest-small/ml-directors-influence_0.csv')

for i in np.arange(1, max_complete_frames+1):
    df = pd.read_csv('/home/ignacio/Datasets/ml-latest-small/ml-directors-influence_'+ str(i) +'.csv')
    df_ml_influences = pd.concat([df_ml_influences, df], ignore_index=True)

print(len(df_ml_influences))

In [ ]:
df_ml_influences.to_csv('/home/ignacio/Datasets/ml-latest-small/ml-directors-influences', index=False)

In [ ]:
df_ml_influences['Influence'] = df_ml_influences['Influence'].str.replace(r'[\[\]\'\']', '')
df_ml_influences = df_ml_influences[df_ml_influences['Influence'].str.len() > 0]
df_ml_influences.head()

In [ ]:
df_ml_influences = df_ml_influences[df_ml_influences['Director'] != '()']
df_ml_influences.head()

In [ ]:
df_ml_influences.to_csv('/home/ignacio/Datasets/ml-latest-small/ml-directors-influences-edges.csv', index=False)

In [ ]:
df_test = df_directors.loc[0:1]
df_test

In [ ]:
df_influenced =df_test.apply(find_influence)
df_influenced

In [ ]:
df_to_save = pd.DataFrame({'Director':df_test, 'Influence':df_influenced})
df_to_save

In [ ]:
df_directors.iloc[0]

In [ ]:
df_directors = df_directors.drop_duplicates()

In [ ]:
len(df_directors)

In [ ]:
df_directors.head()

In [ ]:
df_ml_movies.loc[[7269]]['title_mod'] = df_ml_movies.loc[[7269]]['title']
df_ml_movies.loc[[7269]]

In [ ]:
df['title'] = df['title'].astype('str')

In [ ]:
df['title'].dtype

In [ ]:
df_test.head(20)

In [ ]:
df_ml_movies['director'] = df_ml_movies.apply(find_director, axis=1)

In [ ]:
len(df_ml_movies)

In [ ]:
df_ml_movies = pd.read_csv('/home/ignacio/Datasets/ml-latest-small/movies.csv')
df_ml_movies.head()

In [ ]:
df_genres = df_ml_movies['genres']
df_genres = df_genres.drop_duplicates()
df_genres.head()

In [ ]:
df_genres = df_genres.str.split('|')
df_genres

In [ ]:
genres_values = list(df_genres.values)

In [ ]:
genres = [ g for x in genres_values for g in x ]

In [ ]:
df_genres = pd.DataFrame(data=genres, columns=['genre'])
df_genres.head()

In [ ]:
df_genres = df_genres.drop_duplicates()
df_genres.head()

In [ ]:
df_genres

In [ ]:
df_genres.to_csv('/home/ignacio/Datasets/ml-latest-small/genres.csv', index=False)

In [2]:
df_genres = pd.read_csv('/home/ignacio/Datasets/ml-latest-small/genres.csv')
df_genres

,genre
0,Adventure
1,Animation
2,Children
3,Comedy
4,Fantasy
5,Romance
6,Drama
7,Action
8,Crime
9,Thriller


In [5]:
df_genres['genre'] = df_genres['genre'].str.lower()
df_genres

,genre
0,adventure
1,animation
2,children
3,comedy
4,fantasy
5,romance
6,drama
7,action
8,crime
9,thriller


In [6]:
df_genres.to_csv('/home/ignacio/Datasets/ml-latest-small/genres.csv', index=False)